In [2]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd

## Spotipy setup

- Need client id and client secret which you can get from [here](https://developer.spotify.com/dashboard/login).
- Also need to setup your redirectURI. I just use Google. It doesn't really matter. You can do this by clicking on the green 'Edit Settings' button. Make sure to add and save.
- Username is your user id which you can find by (at least on the desktop app) clicking on your name in the top right to get to your profile, clicking the '...' under your name, hover over 'Share' and navigate to 'Copy Profile Link'. Click it. Paste somewhere to show the link which should be something like https://open.spotify.com/user/** where ** is your user id.

In [4]:
cId = "xxx"
cSecret = "xxx"
redirectURI = "https://google.com/"
username = "xxx"

client_credentials_manager = SpotifyClientCredentials(client_id=cId, client_secret=cSecret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read user-read-currently-playing user-read-playback-state user-read-recently-played'
token = util.prompt_for_user_token(username, scope, cId, cSecret, redirectURI)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

## Importing some saved tracks from spotify to a csv file

This stores the user's saved tracks, artists, and albums into their respective lists.

In [5]:
track_list = []
artist_list = []
album_list = []

song_artist_list = []
song_album_list = []
album_artist_list = []

top_us_50 = sp.user_playlist('spotify', playlist_id='37i9dQZEVXbLRQDuF5jeBp', fields=None)

for t in top_us_50['tracks']['items']:
    track = t['track']
    track_id = track['id']
    track_name = track['name']

    # iterating through each artist of the track
    for artist in track['artists']:
        artist_id = artist['id']

        # storing artist in artist list
        artist_dict = {
            'artist_id': artist_id,
            'artist_name': artist['name']
        }
        artist_list.append(artist_dict)

        # storing track id and artist id in list
        song_artist_dict = {
            'track_id': track_id,
            'artist_id': artist_id
        }
        song_artist_list.append(song_artist_dict)

    # getting album info
    track_album = track['album']
    album_id = track_album['id']
    album_name = track_album['name']
    album_img = track_album['images'][0]['url']
    album_type = track_album['album_type']
    release_date = track_album['release_date']
    total_tracks = track_album['total_tracks']

    # iterating through album artists
    for album in track_album['artists']:
        # storing album id and artist id in list
        album_artist_dict = {
            'album_id': album_id,
            'artist_id': album['id']
        }
        album_artist_list.append(album_artist_dict)

    # storing track id and album id in list
    song_album_dict = {
        'track_id': track_id,
        'album_id': album_id
    }
    song_album_list.append(song_album_dict)

    # storing track info in list
    track_dict = {
        'track_id': track_id,
        'track_name': track_name,
    }
    track_list.append(track_dict)

    # storing album info in list
    album_dict = {
        'album_id': album_id,
        'album_name': album_name,
        'album_img': album_img,
        'album_type': album_type,
        'release_date': release_date,
        'total_tracks': total_tracks
    }
    album_list.append(album_dict)

In [6]:
# Converting the track list into a dataframe

track_df = pd.DataFrame(track_list)
track_df.set_index('track_id', inplace=True) # change index to be the track id
track_df.head() # preview first 5 rows of data

,track_name
track_id,
6ocbgoVGwYJhOv1GgI9NsF,7 rings
4kV4N9D1iKVxx1KLvtTpjS,"break up with your girlfriend, i'm bored"
6MWtB6iiXyIwun0YzU6DFP,Wow.
2JvzF1RMd7lE3KmFlsyZD8,MIDDLE CHILD
3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse


In [7]:
artist_df = pd.DataFrame(artist_list)
artist_df.set_index('artist_id', inplace=True)
artist_df.drop_duplicates(keep='first', inplace=True)

In [8]:
album_df = pd.DataFrame(album_list)
album_df.set_index('album_id', inplace=True)
album_df.drop_duplicates(keep='first', inplace=True) # drop duplicate albums

In [9]:
song_artist_df = pd.DataFrame(song_artist_list)
song_album_df = pd.DataFrame(song_album_list)
album_artist_df = pd.DataFrame(album_artist_list)

Checking dataframe shapes

In [13]:
print('track', track_df.shape)
print('artist', artist_df.shape)
print('album', album_df.shape)
print('song_artist', song_artist_df.shape)
print('song_album', song_album_df.shape)
print('album_artist', album_artist_df.shape)

track (50, 1)
artist (43, 1)
album (36, 5)
song_artist (71, 2)
song_album (50, 2)
album_artist (59, 2)


Saving the dataframes as csv files

In [11]:
track_df.to_csv('songs.csv')
artist_df.to_csv('artists.csv')
album_df.to_csv('albums.csv')

song_artist_df.to_csv('song-artist.csv')
song_album_df.to_csv('song-album.csv')
album_artist_df.to_csv('album-artist.csv')

Check out the csv files!